# Python client for the Glowing Bear data warehouse
---------------

Interacting with the data in the [tranSMART Glowing Bear data warehouse](https://glowingbear.app) via the [REST API](https://glowingbear.app/docs/technical/#transmart-api) and the [Python client](https://github.com/thehyve/transmart-api-client-py) into the Jupyter Notebook analytical environment.

## Getting started

* Make sure you have registered a free account for the [The Hyve public demonstration environment of Glowing Bear](https://glowingbear.app/getting-started/). 
* Make sure you install the Python client with `pip install transmart[full]` to have all functionality available. (When using this notebook in Binder this has already been done for you)
* If you'd like more information or a tailor-made demonstration, please [reach out to The Hyve](https://thehyve.nl/contact/).

First we will import the [tranSMART Python package](https://github.com/thehyve/transmart-api-client-py) by executing the following cell. To execute a cell, select it and press the Run button above.

In [1]:
import transmart as tm

# Connecting to the tranSMART server
Now we will authenticate to the tranSMART server with your credentials. Just execute the following cell, fill in the details and press Enter:

In [2]:
api = tm.get_api(
    host = 'https://transmart.thehyve.net', # URL of tranSMART server connected to your Glowing Bear
    kc_url = "https://keycloak-dwh-test.thehyve.net", # URL of Keycloak connected to your Glowing Bear
    kc_realm = "transmart", # Realm in Keycloak for the tranSMART application
    
    # Keycloak credentials you also use to access Glowing Bear
    user = None, # If None your username will be prompted below
    password = None, # If None your password will be prompted below
    
    print_urls = True # Whether or not to print the API URLs used behind the scenes
)

# Common errors:
# * '401 Client Error: Unauthorized' - Wrong username/password
# * 'HTTPSConnectionPool' - Wrong tranSMART or Keycloak URL or no internet
# * '404 Client Error: Not Found' - Wrong Keycloak realm

Username: ward-demo
KeyCloak password: ········
https://transmart.thehyve.net/v2/studies
https://transmart.thehyve.net/v2/tree_nodes?depth=0&counts=False&tags=True
Existing index cache found. Loaded 10224 tree nodes. Hooray!
https://transmart.thehyve.net/v2/pedigree/relation_types


# Exploring the data
A tranSMART Glowing Bear server can contain multiple data sets or studies, which can be used to control access to per user. Let's show all sets in the server that our user has access to:

In [3]:
studies = api.get_studies()
studies.dataframe.head()

https://transmart.thehyve.net/v2/studies


,bioExperimentId,dimensions,id,secureObjectToken,studyId
0,None,"[study, patient, concept, start time]",2,PUBLIC,SYNTHETICMASS


You will see that each cell prints the corresponding API call that is being done, since we have turned `print_urls` on when connecting. This will give you a better understanding of the [tranSMART Glowing Bear API](https://glowingbear.app/docs/technical/#transmart-api).

The most important data overview in tranSMART Glowing Bear is the tree. Let's see the highest levels of the tree, including patient counts:

In [4]:
tree = api.tree_nodes(depth=2, counts=True)
tree

https://transmart.thehyve.net/v2/tree_nodes?depth=2&counts=True&tags=True


Demographics  (None)/
  Birthdate  (1462)
  Birthplace  (1462)
  Deathdate  (457)
  Ethnicity  (1462)
  Gender  (1462)
  Marital  (1033)
  Race  (1462)
Conditions  (None)/
  ICD10  (None)
  SNOMED  (None)

When we want more details on the tree nodes (like which concept is behind it) we can use the dataframe representation instead:

In [5]:
tree.dataframe.head()

,conceptCode,conceptPath,constraint.conceptCode,constraint.type,fullName,name,observationCount,patientCount,type,visualAttributes
0,NaN,NaN,NaN,NaN,\Demographics\,Demographics,NaN,NaN,UNKNOWN,"[FOLDER, ACTIVE]"
1,Demographics:BIRTHDATE,\Demographics\Birthdate\,Demographics:BIRTHDATE,concept,\Demographics\Birthdate\,Birthdate,1462.0,1462.0,DATE,"[LEAF, ACTIVE, DATE]"
2,Demographics:BIRTHPLACE,\Demographics\Birthplace\,Demographics:BIRTHPLACE,concept,\Demographics\Birthplace\,Birthplace,1462.0,1462.0,CATEGORICAL,"[LEAF, ACTIVE, CATEGORICAL]"
3,Demographics:DEATHDATE,\Demographics\Deathdate\,Demographics:DEATHDATE,concept,\Demographics\Deathdate\,Deathdate,457.0,457.0,DATE,"[LEAF, ACTIVE, DATE]"
4,Demographics:ETHNICITY,\Demographics\Ethnicity\,Demographics:ETHNICITY,concept,\Demographics\Ethnicity\,Ethnicity,1462.0,1462.0,CATEGORICAL,"[LEAF, ACTIVE, CATEGORICAL]"


# Querying for patients and observations
The following example queries for recent, normal pregnancies from parents born in Boston or Cambridge, MA.  
(Later we'll explore in more detail how to build such a query from scratch)

In [6]:
pregnancy_constraint = \
    api.new_constraint(concept='Demographics:BIRTHPLACE', value_list=['Boston MA US', 'Cambridge MA US']) & \
    api.new_constraint(concept='Conditions-SNOMED:72892002', min_start_date='2009-01-01')

api.observations.counts(constraint=pregnancy_constraint)

https://transmart.thehyve.net/v2/observations/counts


{'observationCount': 312, 'patientCount': 18}

With the above call we can quickly see the number of matching patients and the total number of observations (not just pregnancy observations) linked to these subjects, without needing to retrieve the data.

Next, we retrieve the matching patients and show the details for the first five:

In [7]:
api.patients(constraint=pregnancy_constraint).dataframe.head()

https://transmart.thehyve.net/v2/patients


,age,birthDate,deathDate,id,inTrialId,maritalStatus,race,religion,sex,subjectIds.SUBJ_ID,trial
0,None,None,None,551,None,None,None,None,FEMALE,5d761208-d2b4-4569-b4d5-75882501cd83,5d761208-d2b4-4569-b4d5-75882501cd83
1,None,None,None,513,None,None,None,None,FEMALE,57029c04-71c3-4840-8c66-7586c47bfb3f,57029c04-71c3-4840-8c66-7586c47bfb3f
2,None,None,None,954,None,None,None,None,FEMALE,a1a7fa23-7201-4705-b2a1-b423c0a882a2,a1a7fa23-7201-4705-b2a1-b423c0a882a2
3,None,None,None,924,None,None,None,None,FEMALE,9cd7b77b-317d-43c0-baac-1f4637cb9eeb,9cd7b77b-317d-43c0-baac-1f4637cb9eeb
4,None,None,None,363,None,None,None,None,FEMALE,3bda8038-e158-4b07-ab06-42ec8a6a34f7,3bda8038-e158-4b07-ab06-42ec8a6a34f7


Let's also show the first five observations linked to this patient set:

In [8]:
api.observations(constraint=pregnancy_constraint).dataframe.head()

https://transmart.thehyve.net/v2/observations?type=clinical&constraint={"type": "and", "args": [{"type": "subselection", "dimension": "patient", "constraint": {"args": [{"type": "concept", "conceptCode": "Demographics:BIRTHPLACE"}, {"type": "or", "args": [{"type": "value", "valueType": "STRING", "operator": "=", "value": "Boston MA US"}, {"type": "value", "valueType": "STRING", "operator": "=", "value": "Cambridge MA US"}]}], "type": "and"}}, {"type": "subselection", "dimension": "patient", "constraint": {"args": [{"type": "concept", "conceptCode": "Conditions-SNOMED:72892002"}, {"type": "time", "field": {"dimension": "start time", "fieldName": "startDate", "type": "DATE"}, "operator": "->", "values": ["2009-01-01T00:00:00+00:00"]}], "type": "and"}}]}


,concept.conceptCode,concept.conceptPath,concept.name,patient.age,patient.birthDate,patient.deathDate,patient.id,patient.inTrialId,patient.maritalStatus,patient.race,patient.religion,patient.sex,patient.sexCd,patient.subjectIds.SUBJ_ID,patient.trial,start time,stringValue,study.name
0,Conditions-ICD10:J20,\Conditions\ICD10\Acute bronchitis [J20]\,Acute bronchitis [J20],None,None,None,551,None,None,None,None,female,F,5d761208-d2b4-4569-b4d5-75882501cd83,5d761208-d2b4-4569-b4d5-75882501cd83,2011-02-03T00:00:00Z,Acute bronchitis (disorder),SYNTHETICMASS
1,Conditions-ICD10:O15.0,\Conditions\ICD10\Eclampsia [O15]\Eclampsia in...,Eclampsia in pregnancy [O15.0],None,None,None,551,None,None,None,None,female,F,5d761208-d2b4-4569-b4d5-75882501cd83,5d761208-d2b4-4569-b4d5-75882501cd83,2015-12-01T00:00:00Z,Antepartum eclampsia,SYNTHETICMASS
2,Conditions-ICD10:Z34,\Conditions\ICD10\Supervision of normal pregna...,Supervision of normal pregnancy [Z34],None,None,None,551,None,None,None,None,female,F,5d761208-d2b4-4569-b4d5-75882501cd83,5d761208-d2b4-4569-b4d5-75882501cd83,2008-02-12T00:00:00Z,Normal pregnancy,SYNTHETICMASS
3,Conditions-ICD10:Z34,\Conditions\ICD10\Supervision of normal pregna...,Supervision of normal pregnancy [Z34],None,None,None,551,None,None,None,None,female,F,5d761208-d2b4-4569-b4d5-75882501cd83,5d761208-d2b4-4569-b4d5-75882501cd83,2008-10-07T00:00:00Z,Normal pregnancy,SYNTHETICMASS
4,Conditions-ICD10:Z34,\Conditions\ICD10\Supervision of normal pregna...,Supervision of normal pregnancy [Z34],None,None,None,551,None,None,None,None,female,F,5d761208-d2b4-4569-b4d5-75882501cd83,5d761208-d2b4-4569-b4d5-75882501cd83,2015-03-24T00:00:00Z,Normal pregnancy,SYNTHETICMASS


Since we want to reuse this patient set, let's store it on the server and retrieve the returned patient set ID:

In [9]:
patient_set_details = api.create_patient_set(
    name="Recent pregnancies in Boston and Cambridge", constraint=pregnancy_constraint)

patient_set_id = patient_set_details['id']

print(patient_set_id)

https://transmart.thehyve.net/v2/patient_sets?name=Recent pregnancies in Boston and Cambridge
59126


Let's show the most recent stored patient sets linked to my user account, to see our newest included:

In [10]:
patient_sets = api.patient_sets()
patient_sets.dataframe.sort_values(by='id', axis=0).tail()

# Note: Currently only patient sets saved via the API or those used in the Cross Table in Glowing Bear are shown

https://transmart.thehyve.net/v2/patient_sets


,apiVersion,description,errorMessage,id,queryXML,requestConstraints,setSize,status,username
88,v2,Cross table set,None,59122,None,"{""type"":""and"",""args"":[{""type"":""concept"",""conce...",985,FINISHED,74814585-2219-4fd4-8766-11adbb2adbae
89,2.5,Recent pregnancies in Boston and Cambridge,None,59123,None,"{""type"":""and"",""args"":[{""type"":""subselection"",""...",18,FINISHED,74814585-2219-4fd4-8766-11adbb2adbae
90,2.5,Recent pregnancies in Boston and Cambridge,None,59124,None,"{""type"":""and"",""args"":[{""type"":""subselection"",""...",18,FINISHED,74814585-2219-4fd4-8766-11adbb2adbae
91,2.5,Recent pregnancies in Boston and Cambridge,None,59125,None,"{""type"":""and"",""args"":[{""type"":""subselection"",""...",18,FINISHED,74814585-2219-4fd4-8766-11adbb2adbae
92,2.5,Recent pregnancies in Boston and Cambridge,None,59126,None,"{""type"":""and"",""args"":[{""type"":""subselection"",""...",18,FINISHED,74814585-2219-4fd4-8766-11adbb2adbae


We can now use this patient set ID to continue making queries. For example, with the following query that requests only the Ethnicity data for our selected subjects:

In [11]:
one_concept_constraint = api.new_constraint(subject_set_id=patient_set_id, concept='Demographics:ETHNICITY')
api.observations(constraint=one_concept_constraint).dataframe.head()

https://transmart.thehyve.net/v2/observations?type=clinical&constraint={"args": [{"type": "concept", "conceptCode": "Demographics:ETHNICITY"}, {"type": "patient_set", "patientSetId": 59126}], "type": "and"}


,concept.conceptCode,concept.conceptPath,concept.name,patient.age,patient.birthDate,patient.deathDate,patient.id,patient.inTrialId,patient.maritalStatus,patient.race,patient.religion,patient.sex,patient.sexCd,patient.subjectIds.SUBJ_ID,patient.trial,start time,stringValue,study.name
0,Demographics:ETHNICITY,\Demographics\Ethnicity\,Ethnicity,None,None,None,551,None,None,None,None,female,F,5d761208-d2b4-4569-b4d5-75882501cd83,5d761208-d2b4-4569-b4d5-75882501cd83,None,irish,SYNTHETICMASS
1,Demographics:ETHNICITY,\Demographics\Ethnicity\,Ethnicity,None,None,None,1072,None,None,None,None,female,F,b78252ec-a5b0-46e0-bf3f-4cfbae092b10,b78252ec-a5b0-46e0-bf3f-4cfbae092b10,None,irish,SYNTHETICMASS
2,Demographics:ETHNICITY,\Demographics\Ethnicity\,Ethnicity,None,None,None,1194,None,None,None,None,female,F,ce5f7896-1630-439f-888c-204f220a77ff,ce5f7896-1630-439f-888c-204f220a77ff,None,american,SYNTHETICMASS
3,Demographics:ETHNICITY,\Demographics\Ethnicity\,Ethnicity,None,None,None,924,None,None,None,None,female,F,9cd7b77b-317d-43c0-baac-1f4637cb9eeb,9cd7b77b-317d-43c0-baac-1f4637cb9eeb,None,italian,SYNTHETICMASS
4,Demographics:ETHNICITY,\Demographics\Ethnicity\,Ethnicity,None,None,None,954,None,None,None,None,female,F,a1a7fa23-7201-4705-b2a1-b423c0a882a2,a1a7fa23-7201-4705-b2a1-b423c0a882a2,None,polish,SYNTHETICMASS


# Constructing a query
Below we will, step by step, create the query for: _Men with Type 2 Diabetes_.

If a concept is easily found by browsing the tree, like Gender under Demographics, we can start with that:

In [12]:
tree = api.tree_nodes(depth=2, root='\\Demographics\\')
tree.dataframe

https://transmart.thehyve.net/v2/tree_nodes?root=\Demographics\&depth=2&counts=False&tags=True


,conceptCode,conceptPath,constraint.conceptCode,constraint.type,fullName,name,type,visualAttributes
0,NaN,NaN,NaN,NaN,\Demographics\,Demographics,UNKNOWN,"[FOLDER, ACTIVE]"
1,Demographics:BIRTHDATE,\Demographics\Birthdate\,Demographics:BIRTHDATE,concept,\Demographics\Birthdate\,Birthdate,DATE,"[LEAF, ACTIVE, DATE]"
2,Demographics:BIRTHPLACE,\Demographics\Birthplace\,Demographics:BIRTHPLACE,concept,\Demographics\Birthplace\,Birthplace,CATEGORICAL,"[LEAF, ACTIVE, CATEGORICAL]"
3,Demographics:DEATHDATE,\Demographics\Deathdate\,Demographics:DEATHDATE,concept,\Demographics\Deathdate\,Deathdate,DATE,"[LEAF, ACTIVE, DATE]"
4,Demographics:ETHNICITY,\Demographics\Ethnicity\,Demographics:ETHNICITY,concept,\Demographics\Ethnicity\,Ethnicity,CATEGORICAL,"[LEAF, ACTIVE, CATEGORICAL]"
5,Demographics:GENDER,\Demographics\Gender\,Demographics:GENDER,concept,\Demographics\Gender\,Gender,CATEGORICAL,"[LEAF, ACTIVE, CATEGORICAL]"
6,Demographics:MARITAL,\Demographics\Marital\,Demographics:MARITAL,concept,\Demographics\Marital\,Marital,CATEGORICAL,"[LEAF, ACTIVE, CATEGORICAL]"
7,Demographics:RACE,\Demographics\Race\,Demographics:RACE,concept,\Demographics\Race\,Race,CATEGORICAL,"[LEAF, ACTIVE, CATEGORICAL]"


We see that the concept code for Gender is `Demographics:GENDER`, which we will use to start our query constraint:

In [13]:
gender_constraint = api.new_constraint(concept='Demographics:GENDER')
print(type(gender_constraint))
print(gender_constraint)

<class 'transmart.api.v2.constraints.composite.ObservationConstraint'>
{"type": "concept", "conceptCode": "Demographics:GENDER"}


We see that the type of the constraint is an `ObservationConstraint`. And below that we see the representation of our current query.

All parts of our query that need to be true within the same observation will need to be added to the same `ObservationConstraint`. For the _Men_ part of our query both concept Gender and value Male need to be true for the same observation, so they should be together in one `ObservationConstraint`.  
The _Type 2 Diabetes_ will need to be a separate observation for the same patients, so we will later create a separate `ObservationConstraint` for that.

Now, we're not just interested in anyone with a recorded Gender, but only those were the value for this is Male. Let's see what the possible values for Gender are:

In [14]:
api.observations.aggregates_per_concept(constraint=gender_constraint)

https://transmart.thehyve.net/v2/observations/aggregates_per_concept


{'aggregatesPerConcept': {'Demographics:GENDER': {'categoricalValueAggregates': {'nullValueCounts': 0,
    'valueCounts': {'F': 721, 'M': 741}}}}}

We see that the possible values are F and M, where we are interested in the subjects with value M. For this we need to restrict our current constraint with the value M.

We can see all possible parameters for our `ObservationConstraint` below:

In [15]:
api.new_constraint().params

{'concept': transmart.api.v2.constraints.atomic.ConceptCodeConstraint,
 'study': transmart.api.v2.constraints.atomic.StudyConstraint,
 'trial_visit': transmart.api.v2.constraints.atomic.TrialVisitConstraint,
 'min_value': transmart.api.v2.constraints.atomic.MinValueConstraint,
 'max_value': transmart.api.v2.constraints.atomic.MaxValueConstraint,
 'min_date_value': transmart.api.v2.constraints.atomic.MinDateValueConstraint,
 'max_date_value': transmart.api.v2.constraints.atomic.MaxDateValueConstraint,
 'value_list': transmart.api.v2.constraints.atomic.ValueListConstraint,
 'min_start_date': transmart.api.v2.constraints.atomic.StartTimeAfterConstraint,
 'max_start_date': transmart.api.v2.constraints.atomic.StartTimeBeforeConstraint,
 'subject_set_id': transmart.api.v2.constraints.atomic.SubjectSetConstraint}

For restricting our categorical value we need to use `value_list` with a list including the value 'M':

In [16]:
gender_constraint.value_list = ['M']
print(gender_constraint)

{"args": [{"type": "concept", "conceptCode": "Demographics:GENDER"}, {"type": "or", "args": [{"type": "value", "valueType": "STRING", "operator": "=", "value": "M"}]}], "type": "and"}


We can see in the above representation that we are searching for all observations that are both linked to the Concept Gender AND has the value 'M'. We will thus find all observations of Gender male and the patients linked to those observations.

Let's count how many observations and patients match this query:

In [17]:
api.observations.counts(constraint=gender_constraint)

https://transmart.thehyve.net/v2/observations/counts


{'observationCount': 741, 'patientCount': 741}

The patient count indeed represents the number of male subjects in our study.

Note that the observation count is not the the number of ALL observations linked to male patients, but only the observations of concept Gender with Value male.

If we instead want to treat our query as a patient set, and want to know how many observations in total are linked to these patients we have to wrap our query in a patient subselection constraint:

In [18]:
gender_constraint.subselection = 'patient'
print(gender_constraint)

{"type": "subselection", "dimension": "patient", "constraint": {"args": [{"type": "concept", "conceptCode": "Demographics:GENDER"}, {"type": "or", "args": [{"type": "value", "valueType": "STRING", "operator": "=", "value": "M"}]}], "type": "and"}}


We will now see that the observation count for this query is indeed much higher:

In [19]:
api.observations.counts(constraint=gender_constraint)

https://transmart.thehyve.net/v2/observations/counts


{'observationCount': 8930, 'patientCount': 741}

Our first `ObservationConstraint` for the _Male_ part of our question is done. Now, we need to find what the tree node or concept code is for _Type 2 Diabetes_.

We know we have used SNOMED (and ICD10) to structure the data in our tranSMART server. In [Bioportal we find](https://bioportal.bioontology.org/ontologies/SNOMEDCT?p=classes&conceptid=44054006) that the SNOMED code (notation) for 'Type 2 diabetes mellitus' is 44054006. So let's search our tree for that:

In [20]:
nodes_with_snomed_code = api.search_tree_node('name:44054006')
nodes_with_snomed_code

['\\Conditions\\SNOMED\\SNOMED CT Concept [138875005]\\Clinical finding [404684003]\\Disease [64572001]\\Disorder by body site [123946008]\\Disorder of body system [362965005]\\Disorder of endocrine system [362969004]\\Diabetes mellitus [73211009]\\Type 2 diabetes mellitus [44054006]\\',
 '\\Conditions\\SNOMED\\SNOMED CT Concept [138875005]\\Clinical finding [404684003]\\Disease [64572001]\\Metabolic disease [75934005]\\Disorder of carbohydrate metabolism [20957000]\\Disorder of glucose metabolism [126877002]\\Diabetes mellitus [73211009]\\Type 2 diabetes mellitus [44054006]\\',
 '\\Conditions\\SNOMED\\SNOMED CT Concept [138875005]\\Clinical finding [404684003]\\Finding by site [118234003]\\Disorder by body site [123946008]\\Disorder of body system [362965005]\\Disorder of endocrine system [362969004]\\Diabetes mellitus [73211009]\\Type 2 diabetes mellitus [44054006]\\']

We see that three nodes match with the code, since Type 2 diabetes mellitus is placed under multiple parents in the SNOMED ontology tree. Since they are all referring to the same concept, we take the first node and get the node details:

In [21]:
node_details = api.tree_dict[nodes_with_snomed_code[0]]
node_details

{'conceptCode': 'Conditions-SNOMED:44054006',
 'conceptPath': '\\Conditions\\SNOMED\\SNOMED CT Concept [138875005]\\Clinical finding [404684003]\\Finding by site [118234003]\\Disorder by body site [123946008]\\Disorder of body system [362965005]\\Disorder of endocrine system [362969004]\\Diabetes mellitus [73211009]\\Type 2 diabetes mellitus [44054006]\\',
 'constraint.conceptCode': 'Conditions-SNOMED:44054006',
 'constraint.type': 'concept',
 'name': 'Type 2 diabetes mellitus [44054006]',
 'type': 'CATEGORICAL',
 'visualAttributes': ['LEAF', 'ACTIVE', 'CATEGORICAL']}

Here, we learn that the concept code in tranSMART Glowing Bear is `Conditions-SNOMED:44054006`, which we will use to construct our second `ObservationConstraint`:

In [22]:
diabetes_constraint = api.new_constraint(
    concept=node_details['constraint.conceptCode'], subselection='patient')
print(diabetes_constraint)
print()
print(api.observations.counts(constraint=diabetes_constraint))

{"type": "subselection", "dimension": "patient", "constraint": {"type": "concept", "conceptCode": "Conditions-SNOMED:44054006"}}

https://transmart.thehyve.net/v2/observations/counts
{'observationCount': 1764, 'patientCount': 97}


Now we have also finished the second part of our query. We are now looking for the intersection between the two patient sets we have made.

For this we can simply use the `&` or `|` operators and brackets to combine multiple `ObservationConstraints` into one `GroupConstraint`:

In [23]:
men_with_diabetes_constraint = gender_constraint & diabetes_constraint
print(type(men_with_diabetes_constraint))
print()
print(men_with_diabetes_constraint)

<class 'transmart.api.v2.constraints.composite.GroupConstraint'>

{"type": "and", "args": [{"type": "subselection", "dimension": "patient", "constraint": {"args": [{"type": "concept", "conceptCode": "Demographics:GENDER"}, {"type": "or", "args": [{"type": "value", "valueType": "STRING", "operator": "=", "value": "M"}]}], "type": "and"}}, {"type": "subselection", "dimension": "patient", "constraint": {"type": "concept", "conceptCode": "Conditions-SNOMED:44054006"}}]}


Side note: When creating a `GroupConstraint`, the multiple `ObservationConstraint`s are automatically each wrapped with a patient subselection. So technically it wasn't necessary that we did this ourselves above for the two constraints.

Now all that's left is for us to count our final query and retrieve the patients and observations of interest!

In [24]:
print(api.observations.counts(constraint=men_with_diabetes_constraint))
display(api.patients(constraint=men_with_diabetes_constraint).dataframe.head())
display(api.observations(constraint=men_with_diabetes_constraint).dataframe.head())

https://transmart.thehyve.net/v2/observations/counts
{'observationCount': 805, 'patientCount': 44}
https://transmart.thehyve.net/v2/patients


,age,birthDate,deathDate,id,inTrialId,maritalStatus,race,religion,sex,subjectIds.SUBJ_ID,trial
0,None,None,None,62,None,None,None,None,MALE,093c5b43-9b43-40a4-8cb6-ba2292cba7c0,093c5b43-9b43-40a4-8cb6-ba2292cba7c0
1,None,None,None,158,None,None,None,None,MALE,1a2305e5-686d-4fc1-9c25-691b63dadcab,1a2305e5-686d-4fc1-9c25-691b63dadcab
2,None,None,None,231,None,None,None,None,MALE,27d3c718-96df-465f-b0de-227b958a3b67,27d3c718-96df-465f-b0de-227b958a3b67
3,None,None,None,245,None,None,None,None,MALE,29c6eb5a-27c0-49e5-8814-c71b153c5226,29c6eb5a-27c0-49e5-8814-c71b153c5226
4,None,None,None,303,None,None,None,None,MALE,32342351-2ce8-4fd3-a721-fbcd033cd3a9,32342351-2ce8-4fd3-a721-fbcd033cd3a9


https://transmart.thehyve.net/v2/observations?type=clinical&constraint={"type": "and", "args": [{"type": "subselection", "dimension": "patient", "constraint": {"args": [{"type": "concept", "conceptCode": "Demographics:GENDER"}, {"type": "or", "args": [{"type": "value", "valueType": "STRING", "operator": "=", "value": "M"}]}], "type": "and"}}, {"type": "subselection", "dimension": "patient", "constraint": {"type": "concept", "conceptCode": "Conditions-SNOMED:44054006"}}]}


,concept.conceptCode,concept.conceptPath,concept.name,patient.age,patient.birthDate,patient.deathDate,patient.id,patient.inTrialId,patient.maritalStatus,patient.race,patient.religion,patient.sex,patient.sexCd,patient.subjectIds.SUBJ_ID,patient.trial,start time,stringValue,study.name
0,Conditions-ICD10:J20,\Conditions\ICD10\Acute bronchitis [J20]\,Acute bronchitis [J20],None,None,None,1178,None,None,None,None,male,M,cb653c5e-8f76-4568-b475-7a969819672a,cb653c5e-8f76-4568-b475-7a969819672a,2009-10-11T00:00:00Z,Acute bronchitis (disorder),SYNTHETICMASS
1,Conditions-ICD10:J20,\Conditions\ICD10\Acute bronchitis [J20]\,Acute bronchitis [J20],None,None,None,1178,None,None,None,None,male,M,cb653c5e-8f76-4568-b475-7a969819672a,cb653c5e-8f76-4568-b475-7a969819672a,2015-10-21T00:00:00Z,Acute bronchitis (disorder),SYNTHETICMASS
2,Conditions-ICD10:I64,"\Conditions\ICD10\Stroke, not specified as hae...","Stroke, not specified as haemorrhage or infarc...",None,None,None,1178,None,None,None,None,male,M,cb653c5e-8f76-4568-b475-7a969819672a,cb653c5e-8f76-4568-b475-7a969819672a,2014-01-15T00:00:00Z,Stroke,SYNTHETICMASS
3,Conditions-ICD10:E11,\Conditions\ICD10\Type 2 diabetes mellitus [E11]\,Type 2 diabetes mellitus [E11],None,None,None,1178,None,None,None,None,male,M,cb653c5e-8f76-4568-b475-7a969819672a,cb653c5e-8f76-4568-b475-7a969819672a,1998-05-08T00:00:00Z,Diabetes,SYNTHETICMASS
4,Conditions-SNOMED:10509002,\Conditions\SNOMED\SNOMED CT Concept [13887500...,Acute bronchitis [10509002],None,None,None,1178,None,None,None,None,male,M,cb653c5e-8f76-4568-b475-7a969819672a,cb653c5e-8f76-4568-b475-7a969819672a,2009-10-11T00:00:00Z,Acute bronchitis (disorder),SYNTHETICMASS
